<a href="https://colab.research.google.com/github/gilvandrocesardemedeiros/CircuitosDigitais/blob/master/Replace_Missing_Values_Temperature%5BAracaju%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importando bibliotecas
import pandas as pd
import numpy as np
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
#Leitura de dados
dataSet = pd.read_csv("https://raw.githubusercontent.com/gilvandrocesardemedeiros/Meteorology_replace-missing-values/master/DadosDiarios/aracajudiario.csv",
                      sep=';',skiprows=16)

In [0]:
#Transformando data em variável do tipo datetime
dataSet["Data"] = pd.to_datetime(dataSet["Data"], format = "%d/%m/%Y")

In [4]:
#Descrevendo base de dados
dataSet.describe()

,Estacao,Hora,Precipitacao,TempMaxima,TempMinima,Insolacao,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media,Unnamed: 11
count,22441.0,22441.000000,11198.000000,11210.000000,10076.000000,10767.000000,6743.000000,11071.000000,11196.000000,9385.000000,0.0
mean,83096.0,599.492001,3.344731,29.388662,23.116485,7.837606,3.931915,26.381604,76.868502,2.762278,NaN
std,0.0,600.013154,9.240869,1.402776,1.429518,3.128315,1.309769,1.263401,5.708472,1.094090,NaN
min,83096.0,0.000000,0.000000,23.900000,11.000000,0.000000,0.000000,19.600000,44.250000,0.000000,NaN
25%,83096.0,0.000000,0.000000,28.300000,22.200000,6.200000,3.100000,25.520000,73.250000,2.200000,NaN
50%,83096.0,0.000000,0.000000,29.500000,23.300000,8.900000,4.000000,26.520000,76.500000,2.733333,NaN
75%,83096.0,1200.000000,2.500000,30.400000,24.100000,10.200000,4.700000,27.320000,80.000000,3.400000,NaN
max,83096.0,1200.000000,157.000000,35.300000,27.100000,12.500000,18.100000,29.920000,100.000000,6.900000,NaN


In [5]:
#Verificando dados
dataSet.head()

,Estacao,Data,Hora,Precipitacao,TempMaxima,TempMinima,Insolacao,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media,Unnamed: 11
0,83096,1980-01-01,0,NaN,29.0,NaN,4.1,2.2,26.34,75.5,4.033333,NaN
1,83096,1980-01-01,1200,0.0,NaN,22.5,NaN,NaN,NaN,NaN,NaN,NaN
2,83096,1980-01-02,0,NaN,29.1,NaN,3.9,3.0,26.90,81.5,4.333333,NaN
3,83096,1980-01-02,1200,0.0,NaN,24.3,NaN,NaN,NaN,NaN,NaN,NaN
4,83096,1980-01-03,0,NaN,29.0,NaN,0.3,2.5,26.94,86.5,4.400000,NaN


In [0]:
#Atribuindo "Data" como índice para a base de dados
dataSet = dataSet.set_index("Data")

In [0]:
#Separando os dados em dois dataframes, um para as 00:00 h e outro para as 12:00 h
dataSet00, dataSet12 = dataSet[dataSet["Hora"] == 0], dataSet[dataSet["Hora"] == 1200]

In [0]:
#Descartando as colunas "Hora", "Estacao" e "Unnamed: 11" 
dataSet00, dataSet12 = dataSet00.drop(columns = ["Hora", "Estacao", "Unnamed: 11"]), dataSet12.drop(columns = ["Hora", "Estacao", "Unnamed: 11"])

In [9]:
#Verificando dataframe para os dados disponibilizados às 00:00 h
dataSet00.describe()

,Precipitacao,TempMaxima,TempMinima,Insolacao,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media
count,0.0,11210.000000,0.0,10767.000000,6743.000000,11071.000000,11196.000000,9385.000000
mean,NaN,29.388662,NaN,7.837606,3.931915,26.381604,76.868502,2.762278
std,NaN,1.402776,NaN,3.128315,1.309769,1.263401,5.708472,1.094090
min,NaN,23.900000,NaN,0.000000,0.000000,19.600000,44.250000,0.000000
25%,NaN,28.300000,NaN,6.200000,3.100000,25.520000,73.250000,2.200000
50%,NaN,29.500000,NaN,8.900000,4.000000,26.520000,76.500000,2.733333
75%,NaN,30.400000,NaN,10.200000,4.700000,27.320000,80.000000,3.400000
max,NaN,35.300000,NaN,12.500000,18.100000,29.920000,100.000000,6.900000


In [10]:
#Verificando dataframe para os dados disponibilizados às 12:00 h
dataSet12.describe()

,Precipitacao,TempMaxima,TempMinima,Insolacao,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media
count,11198.000000,0.0,10076.000000,0.0,0.0,0.0,0.0,0.0
mean,3.344731,NaN,23.116485,NaN,NaN,NaN,NaN,NaN
std,9.240869,NaN,1.429518,NaN,NaN,NaN,NaN,NaN
min,0.000000,NaN,11.000000,NaN,NaN,NaN,NaN,NaN
25%,0.000000,NaN,22.200000,NaN,NaN,NaN,NaN,NaN
50%,0.000000,NaN,23.300000,NaN,NaN,NaN,NaN,NaN
75%,2.500000,NaN,24.100000,NaN,NaN,NaN,NaN,NaN
max,157.000000,NaN,27.100000,NaN,NaN,NaN,NaN,NaN


In [0]:
#Eliminando colunas completas por "NaN" em cada uma das bases de dados
dataSet00 = dataSet00.drop(columns = ["Precipitacao","TempMinima"])
dataSet12 = dataSet12.drop(columns = ["TempMaxima","Insolacao","Evaporacao Piche","Temp Comp Media","Umidade Relativa Media","Velocidade do Vento Media"])

In [12]:
#Exibindo a base de dados recém filtrada para 00:00 h
dataSet00.head()

,TempMaxima,Insolacao,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media
Data,,,,,,
1980-01-01,29.0,4.1,2.2,26.34,75.50,4.033333
1980-01-02,29.1,3.9,3.0,26.90,81.50,4.333333
1980-01-03,29.0,0.3,2.5,26.94,86.50,4.400000
1980-01-04,28.9,3.0,1.8,25.36,87.50,3.566667
1980-01-05,28.8,1.2,1.5,26.02,83.25,2.166667


In [13]:
#Exibindo a base de dados recém filtrada para 12:00 h
dataSet12.head()

,Precipitacao,TempMinima
Data,,
1980-01-01,0.0,22.5
1980-01-02,0.0,24.3
1980-01-03,0.0,25.2
1980-01-04,12.2,25.1
1980-01-05,8.2,21.9


In [0]:
#Criando o intervalo completo de tempo de 01-01-1980 à 31-12-2017
dataInicial = '1980-01-01'
dataFinal = '2017-12-31'
tempo = pd.date_range(dataInicial, dataFinal)
#Atribuindo este intervalo de tempo à um dataSet provisório
dataSetProv = pd.DataFrame()
dataSetProv["Data"] = tempo

In [0]:
#Atribuindo o índice para o dataSet provisório como sendo a coluna de datas
dataSetProv = dataSetProv.set_index("Data")

In [16]:
#Visualizando o dataSet provisório
dataSetProv.head()

""
Data
1980-01-01
1980-01-02
1980-01-03
1980-01-04
1980-01-05


In [0]:
#Mesclando o dataSet provisório como sendo o resultado da junção dos dataSet00 e dataSet12
dataSetProv = dataSetProv.join(dataSet00).join(dataSet12)

In [18]:
#Mostrando o resultado da junção
dataSetProv.head()

,TempMaxima,Insolacao,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media,Precipitacao,TempMinima
Data,,,,,,,,
1980-01-01,29.0,4.1,2.2,26.34,75.50,4.033333,0.0,22.5
1980-01-02,29.1,3.9,3.0,26.90,81.50,4.333333,0.0,24.3
1980-01-03,29.0,0.3,2.5,26.94,86.50,4.400000,0.0,25.2
1980-01-04,28.9,3.0,1.8,25.36,87.50,3.566667,12.2,25.1
1980-01-05,28.8,1.2,1.5,26.02,83.25,2.166667,8.2,21.9


In [19]:
#Quantidade de dias existentes na base de dados
len(dataSetProv)

13880

In [0]:
#Tornando o dataSet como sendo o dataSet provisório
dataSet = dataSetProv

In [21]:
#Descrevendo o novo dataSet
dataSet.describe()

,TempMaxima,Insolacao,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media,Precipitacao,TempMinima
count,11210.000000,10767.000000,6743.000000,11071.000000,11196.000000,9385.000000,11198.000000,10076.000000
mean,29.388662,7.837606,3.931915,26.381604,76.868502,2.762278,3.344731,23.116485
std,1.402776,3.128315,1.309769,1.263401,5.708472,1.094090,9.240869,1.429518
min,23.900000,0.000000,0.000000,19.600000,44.250000,0.000000,0.000000,11.000000
25%,28.300000,6.200000,3.100000,25.520000,73.250000,2.200000,0.000000,22.200000
50%,29.500000,8.900000,4.000000,26.520000,76.500000,2.733333,0.000000,23.300000
75%,30.400000,10.200000,4.700000,27.320000,80.000000,3.400000,2.500000,24.100000
max,35.300000,12.500000,18.100000,29.920000,100.000000,6.900000,157.000000,27.100000


In [22]:
#Mostrando o cabeçalho do novo dataSet
dataSet.head()

,TempMaxima,Insolacao,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media,Precipitacao,TempMinima
Data,,,,,,,,
1980-01-01,29.0,4.1,2.2,26.34,75.50,4.033333,0.0,22.5
1980-01-02,29.1,3.9,3.0,26.90,81.50,4.333333,0.0,24.3
1980-01-03,29.0,0.3,2.5,26.94,86.50,4.400000,0.0,25.2
1980-01-04,28.9,3.0,1.8,25.36,87.50,3.566667,12.2,25.1
1980-01-05,28.8,1.2,1.5,26.02,83.25,2.166667,8.2,21.9


# Modelagem para previsão de Temperatura Máxima

In [0]:
#Separando base de dados para treino de rede neural para previsão de temperatura máxima
dataTreinoTempMax = dataSet.dropna(subset = ["TempMaxima"])

In [0]:
#Separando a variável que será prevista, TempMaxima, da base de dados
tempMax = dataTreinoTempMax["TempMaxima"]
dataTreinoTempMax = dataTreinoTempMax.drop(columns = "TempMaxima")

In [25]:
#Visualizando cabeçalho dos dados para treino
dataTreinoTempMax.head()

,Insolacao,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media,Precipitacao,TempMinima
Data,,,,,,,
1980-01-01,4.1,2.2,26.34,75.50,4.033333,0.0,22.5
1980-01-02,3.9,3.0,26.90,81.50,4.333333,0.0,24.3
1980-01-03,0.3,2.5,26.94,86.50,4.400000,0.0,25.2
1980-01-04,3.0,1.8,25.36,87.50,3.566667,12.2,25.1
1980-01-05,1.2,1.5,26.02,83.25,2.166667,8.2,21.9


In [26]:
#Visualizando cabeçalho das informações de temperatura 
tempMax.head()

Data
1980-01-01    29.0
1980-01-02    29.1
1980-01-03    29.0
1980-01-04    28.9
1980-01-05    28.8
Name: TempMaxima, dtype: float64

In [0]:
#Guardando a informação de como as colunas estão distribuídas no dataFrame
colunas = dataTreinoTempMax.columns

In [0]:
#Efetuando uma transformação que lineariza todas as informações para cada coluna entre -1 e 1
mms = MinMaxScaler(feature_range=(-1, 1))
dataTreinoTempMax = mms.fit_transform(dataTreinoTempMax)

In [29]:
#Visualizando como está a base de dados para treino após a linearização
prov = pd.DataFrame(dataTreinoTempMax)
prov.columns = colunas
prov.describe()

,Insolacao,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media,Precipitacao,TempMinima
count,10749.000000,6733.000000,11064.000000,11183.000000,9365.000000,11178.000000,10065.000000
mean,0.254838,-0.565322,0.314089,0.170119,-0.198969,-0.957337,0.505041
std,0.499799,0.144675,0.244769,0.204843,0.316555,0.117807,0.177576
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,-0.008000,-0.657459,0.147287,0.040359,-0.362319,-1.000000,0.391304
50%,0.424000,-0.558011,0.341085,0.156951,-0.207730,-1.000000,0.527950
75%,0.632000,-0.480663,0.496124,0.282511,-0.014493,-0.966879,0.627329
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
#Trocando dados faltantes por 0 
dataTreinoTempMax = np.nan_to_num(dataTreinoTempMax, copy = False)

In [31]:
#Verificando dimensões do dataSet
print(dataTreinoTempMax.shape)

(11210, 7)


In [0]:
#Separando conjunto de treino e teste para previsão de temperatura máxima (90% para treino e 10% para teste)
X_trainTempMax, X_testTempMax, Y_trainTempMax, Y_testTempMax = train_test_split(dataTreinoTempMax, tempMax, test_size = 0.1, random_state = 9)

In [0]:
#Definindo uma função para criar a MLP
def trainMLP(optimizer='adam', init='random_uniform', units = 7, camadas = 2):    
  #Criando o modelo
  mlp = Sequential()
  #Adicionando a camada de entrada
  mlp.add(Dense(activation = 'tanh', input_dim = units, units = units, kernel_initializer = init, bias_initializer='zeros'))
  #Adicionando as camadas escondidas
  for i in range(camadas):
    mlp.add(Dense(activation = 'tanh', units = units, kernel_initializer = init, bias_initializer='zeros'))
  #Adicionando a camada de saída
  mlp.add(Dense(units = 1, kernel_initializer = init, bias_initializer='zeros'))    
  # Compilando o modelo
  mlp.compile(loss='mean_squared_error', optimizer = optimizer, metrics= ['mae'])
  return mlp

In [0]:
#Modelo de classificação do Keras
estimator = KerasRegressor(build_fn=trainMLP, epochs = 200, verbose = 2)

In [0]:
#Criando o objeto para Cross Validation
kfold = KFold(n_splits=4, shuffle=True, random_state=5)

In [0]:
#Parâmetros para o Grid Search
parameters = {'batch_size': [10],             
              'camadas': [2],
              'units': [7]}

In [0]:
#Modelagem para buscar os melhores parâmetros
grid_searchTempMax = GridSearchCV(estimator = estimator, param_grid = parameters,
                     scoring = "neg_mean_squared_error", verbose=500, cv = kfold, return_train_score = True)

In [0]:
#Busca pelos melhores parâmetros
grid_resultTempMax = grid_searchTempMax.fit(X_trainTempMax, Y_trainTempMax)

Fitting 4 folds for each of 1 candidates, totalling 4 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] batch_size=10, camadas=2, units=7 ...............................
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/200
 - 1s - loss: 657.5869 - mean_absolute_error: 25.5131
Epoch 2/200
 - 1s - loss: 381.5397 - mean_absolute_error: 19.4223
Epoch 3/200
 - 1s - loss: 212.1122 - mean_absolute_error: 14.4348
Epoch 4/200
 - 1s - loss: 105.3986 - mean_absolute_error: 10.1059
Epoch 5/200
 - 1s - loss: 44.4577 - mean_absolute_error: 6.4509
Epoch 6/200
 - 1s - loss: 15.2054 - mean_absolute_error: 3.5849
Epoch 7/200
 - 1s - loss: 4.6586 - mean_absolute_error: 1.8004
Epoch 8/200
 - 1s - loss: 2.2342 - mean_absolute_error: 1.2352
Epoch 9/200
 - 1s - loss: 1.9564 - mean_absolute_error: 1.1527
Epoch 10/200
 - 1s - loss: 1.9447 - mean_absolute_error: 1.1465
Epoch 11/200
 - 1s - los

In [0]:
#Exibição dos melhores resultados
print("Melhor resultado: %f, usando %s" % (grid_resultTempMax.best_score_, grid_resultTempMax.best_params_))
means = grid_resultTempMax.cv_results_['mean_test_score']
stds = grid_resultTempMax.cv_results_['std_test_score']
params = grid_resultTempMax.cv_results_['params']
print("Resultados (Desvio padrão) {Parâmetros}")
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) %r" % (mean, stdev, param))

In [0]:
#Gráfico de previsão para o conjunto de treino
plt.scatter(grid_searchTempMax.predict(X_trainTempMax), Y_trainTempMax)
plt.title("Temperatura máxima estimada para o conjunto de treino")
plt.xlabel("Temperatura estimada")
plt.ylabel("Temperatura real")
plt.show()

In [0]:
#Mostrando uma matriz de correlação das variáveis de treino com a melhor modelagem
print(np.corrcoef(grid_searchTempMax.predict(X_trainTempMax), Y_trainTempMax))

In [0]:
#Previsão com base no resultado da Grid Search
Y_predTempMax = grid_searchTempMax.predict(X_testTempMax)

In [0]:
#Gráfico que mostra a relação entre a temperatura máxima real e a prevista para o conjunto de teste
plt.scatter(Y_predTempMax, Y_testTempMax)
plt.title("Temperatura máxima estimada para o conjunto de teste")
plt.xlabel("Temperatura estimada")
plt.ylabel("Temperatura real")
plt.show()

In [0]:
#Mostrando uma matriz de correlação das variáveis de teste com a melhor modelagem
print(np.corrcoef(Y_predTempMax, Y_testTempMax))

In [0]:
#Erro absoluto médio entre a temperatura máxima prevista e a real
print(mean_absolute_error(Y_testTempMax, Y_predTempMax))

In [0]:
#Erro quadrático médio entre a temperatura máxima prevista e a real
print(mean_squared_error(Y_testTempMax, Y_predTempMax))

# Modelagem para previsão de Temperatura Mínima

In [0]:
#Separando dados para treinar rede neural para previsão de temperatura mínima
dataTreinoTempMin = dataSet.dropna(subset = ["TempMinima"])

In [0]:
#Separando a variável que será prevista, TempMinima, da base de dados
tempMin = dataTreinoTempMin["TempMinima"]
dataTreinoTempMin = dataTreinoTempMin.drop(columns = "TempMinima")

In [0]:
#Visualizando dados de treino para temperatura mínima
dataTreinoTempMin.head()

In [0]:
#Visualizando informações de temperatura mínima para cada um dos dados de treino 
tempMin.head()

In [0]:
#Guardando informação de como as colunas estão distribuídas no dataFrame 
colunas = dataTreinoTempMin.columns

In [0]:
#Efetuando uma transformação que lineariza todas as informações para cada coluna entre -1 e 1
mms = MinMaxScaler(feature_range=(-1, 1))
dataTreinoTempMin = mms.fit_transform(dataTreinoTempMin)

In [0]:
#Visualizando como está a base de dados para treino após a linearização
prov = pd.DataFrame(dataTreinoTempMin)
prov.columns = colunas
prov.describe()

In [0]:
#Trocando dados faltantes por 0
dataTreinoTempMin = np.nan_to_num(dataTreinoTempMin, copy = False)

In [0]:
#Verificando dimensões do dataSet
print(dataTreinoTempMin.shape)

In [0]:
#Separando conjunto de treino e teste para previsão de temperatura máxima (90% para treino e 10% para teste)
X_trainTempMin, X_testTempMin, Y_trainTempMin, Y_testTempMin = train_test_split(dataTreinoTempMin, tempMin, test_size = 0.1, random_state = 9)

In [0]:
#Modelo de classificação do Keras
estimator = KerasRegressor(build_fn=trainMLP, epochs = 200, verbose = 2)

In [0]:
#Criando o objeto para Cross Validation
kfold = KFold(n_splits=4, shuffle=True, random_state=5)

In [0]:
#Parâmetros para o Grid Search
parameters = {'batch_size': [10],             
              'camadas': [2],
              'units': [7]}

In [0]:
#Modelagem para buscar os melhores parâmetros
grid_searchTempMin = GridSearchCV(estimator = estimator, param_grid = parameters,
                     scoring = "neg_mean_squared_error", verbose=500, cv = kfold, return_train_score = True)

In [0]:
#Busca pelos melhores parâmetros
grid_resultTempMin = grid_searchTempMin.fit(X_trainTempMin, Y_trainTempMin)

In [0]:
#Exibição dos melhores resultados
print("Melhor resultado: %f, usando %s" % (grid_resultTempMin.best_score_, grid_resultTempMin.best_params_))
means = grid_resultTempMin.cv_results_['mean_test_score']
stds = grid_resultTempMin.cv_results_['std_test_score']
params = grid_resultTempMin.cv_results_['params']
print("Resultados (Desvio padrão) {Parâmetros}")
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) %r" % (mean, stdev, param))

In [0]:
#Gráfico de previsão para o conjunto de treino
plt.scatter(grid_searchTempMin.predict(X_trainTempMin), Y_trainTempMin)
plt.title("Temperatura mínima estimada para o conjunto de treino")
plt.xlabel("Temperatura estimada")
plt.ylabel("Temperatura real")
plt.show()

In [0]:
#Mostrando uma matriz de correlação das variáveis de treino com a melhor modelagem
print(np.corrcoef(grid_searchTempMin.predict(X_trainTempMin), Y_trainTempMin))

In [0]:
#Previsão para os dados de teste a partir da modelagem obtida
Y_pred = grid_searchTempMin.predict(X_testTempMin)

In [0]:
#Gráfico que mostra a relação entre a temperatura mínima real e a temperatura mínima prevista para os dados de teste
plt.scatter(Y_predTempMin, Y_testTempMin)
plt.title("Temperatura mínima estimada para o conjunto de teste")
plt.xlabel("Temperatura estimada")
plt.ylabel("Temperatura real")
plt.show()

In [0]:
#Mostrando uma matriz de correlação das variáveis de teste com a melhor modelagem
print(np.corrcoef(Y_predTempMin, Y_testTempMin))

In [0]:
#Erro absoluto médio entre temperatura mínima real e prevista para os dados de teste
print(mean_absolute_error(Y_testTempMin, Y_predTempMin))

In [0]:
#Erro quadrático médio entre a temperatura mínima real e a prevista para os dados de teste
print(mean_squared_error(Y_testTempMin, Y_predTempMin))